In [1]:
!pip install -q peft transformers datasets

In [2]:
from datasets import load_dataset

ds = load_dataset("food101")

Generating train split:   0%|          | 0/75750 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/25250 [00:00<?, ? examples/s]

In [3]:
labels = ds["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

id2label[2]

'baklava'

### 1. Chuẩn bị dữ liệu

In [4]:
from transformers import AutoImageProcessor

image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")

2024-07-27 11:11:37.263402: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-27 11:11:37.263544: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-27 11:11:37.414197: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


 - Chuẩn Hóa: Đảm bảo giá trị tensor ảnh được chuẩn hóa đúng cách dựa trên thống kê của tập dữ liệu.
 - Biến Đổi Huấn Luyện: Bao gồm tăng cường dữ liệu (cắt ngẫu nhiên và lật theo chiều ngang) để giúp mô hình tổng quát hơn.
 - Biến Đổi Kiểm Tra: Áp dụng thay đổi kích thước và cắt trung tâm mà không có tăng cường dữ liệu để đánh giá mô hình.
 - Hàm Tiền Xử Lý: Áp dụng các phép biến đổi đã định nghĩa cho các lô ảnh, chuyển đổi chúng thành tensor và chuẩn hóa chúng.

In [5]:
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
train_transforms = Compose(
    [
        RandomResizedCrop(image_processor.size["height"]),
        RandomHorizontalFlip(),
        ToTensor(),
        normalize,
    ]
)

val_transforms = Compose(
    [
        Resize(image_processor.size["height"]),
        CenterCrop(image_processor.size["height"]),
        ToTensor(),
        normalize,
    ]
)

def preprocess_train(example_batch):
    example_batch["pixel_values"] = [train_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch

def preprocess_val(example_batch):
    example_batch["pixel_values"] = [val_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch

In [6]:
train_ds = ds["train"]
val_ds = ds["validation"]

train_ds.set_transform(preprocess_train)
val_ds.set_transform(preprocess_val)

In [7]:
import torch

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

### 2. Đào tạo mô hình

Sử dụng mô hình google/vit-base-patch16-224-in21k, nhưng bạn có thể sử dụng bất kỳ mô hình phân loại ảnh nào bạn muốn. Truyền các từ điển label2id và id2label vào mô hình để nó biết cách ánh xạ các nhãn số nguyên thành nhãn lớp của chúng.

In [8]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True,
)

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Mỗi phương pháp **PEFT** (Parameter-Efficient Fine-Tuning) yêu cầu một cấu hình để lưu trữ tất cả các tham số chỉ định cách mà phương pháp PEFT sẽ được áp dụng. Khi cấu hình đã được thiết lập, hãy truyền nó vào hàm `get_peft_model()` cùng với mô hình cơ sở để tạo ra một `PeftModel` có thể huấn luyện được. **LoRA** (Low-Rank Adaptation) phân rã ma trận cập nhật trọng số thành hai ma trận nhỏ hơn. Kích thước của các ma trận thấp rank này được xác định bởi rank hoặc `r`. Rank cao hơn có nghĩa là mô hình có nhiều tham số hơn để huấn luyện, nhưng cũng đồng nghĩa với việc mô hình có khả năng học tập tốt hơn. Bạn cũng nên chỉ định các `target_modules` để xác định nơi các ma trận nhỏ hơn sẽ được chèn vào. Trong hướng dẫn này, bạn sẽ nhắm vào các ma trận truy vấn và giá trị của các khối attention. Các tham số quan trọng khác cần thiết lập là `lora_alpha` (hệ số tỷ lệ), `bias` (liệu không có, tất cả hoặc chỉ các tham số LoRA bias nên được huấn luyện), và `modules_to_save` (các mô-đun ngoài các lớp LoRA sẽ được huấn luyện và lưu). Tất cả các tham số này - và nhiều hơn nữa - được tìm thấy trong `LoraConfig`.

In [9]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="none",
    modules_to_save=["classifier"],
)
model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 667,493 || all params: 86,543,818 || trainable%: 0.7713


In [10]:
from transformers import TrainingArguments, Trainer

peft_model_id = f"loRA"
batch_size = 128

args = TrainingArguments(
    peft_model_id,
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-3,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    fp16=True,
    num_train_epochs=5,
    logging_steps=10,
    load_best_model_at_end=True,
    label_names=["labels"],
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=image_processor,
    data_collator=collate_fn,
)
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


UsageError: api_key not configured (no-tty). call wandb.login(key=[your_api_key])

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.push_to_hub("loRA")

In [ ]:
from peft import PeftConfig, PeftModel
from transformers import AutoImageProcessor
from PIL import Image
import requests

config = PeftConfig.from_pretrained("FuuToru/loRA")
model = AutoModelForImageClassification.from_pretrained(
    config.base_model_name_or_path,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True,
)
model = PeftModel.from_pretrained(model, "stevhliu/vit-base-patch16-224-in21k-lora")

url = "https://huggingface.co/datasets/sayakpaul/sample-datasets/resolve/main/beignets.jpeg"
image = Image.open(requests.get(url, stream=True).raw)
image

In [ ]:
encoding = image_processor(image.convert("RGB"), return_tensors="pt")

In [ ]:
with torch.no_grad():
    outputs = model(**encoding)
    logits = outputs.logits

predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])
"Predicted class: beignets"